TensorFlow Keras



In [2]:
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [5]:
X, y = make_classification(
    n_samples=5000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    n_classes=3,
    weights=[1/3]*3, # hacemos 3 clases balanceadas
    random_state=42

)

df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(1,21)])
df['class'] = y
df.head()



,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,class
0,0.577910,-1.175159,3.182231,0.404955,-0.613865,-1.039464,-0.239075,-2.836809,-8.026848,-9.041112,...,-3.014423,-0.923978,1.094650,0.711980,1.824328,2.145572,0.051517,-3.797374,1.102314,0
1,-3.256100,0.919359,1.291473,0.030575,1.660010,-0.552647,0.659944,-1.471791,0.089973,-0.870679,...,-3.269361,0.624425,0.528576,-2.705067,0.700740,-2.075426,-1.278485,0.824840,-0.712165,1
2,0.263308,-1.464582,1.580419,0.452207,0.693424,-0.664855,-2.134743,-3.664896,-3.941314,-4.077946,...,-2.652247,0.734020,-0.504239,-1.229404,1.276939,0.018643,-1.459663,-2.030924,-2.562835,0
3,2.327166,2.972473,2.546896,-0.887235,0.763177,2.039143,-1.942751,-0.079054,4.994518,-3.459207,...,-4.096714,2.801948,-0.499836,0.922420,0.170235,2.051526,1.737861,1.805639,2.357027,0
4,0.047870,0.033218,-1.179213,0.779208,1.454919,2.839628,1.145734,2.988370,6.476426,8.485573,...,1.315190,0.065581,1.876118,-0.038665,-2.712738,0.374466,-0.720040,0.809654,-2.902803,1


In [6]:

X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)